# Modelado

## Funciones para entrenamiento y testeo

In [ ]:
def grid_search_function(model, param_grid, X_train, y_train, pca_components=[0.50, 0.99], cv=2):
    """
    Realiza una búsqueda en cuadrícula (grid search) con validación cruzada
    para encontrar los mejores parámetros de un modelo de aprendizaje automático.

    Parámetros:
    - model: El modelo de aprendizaje automático a utilizar.
    - param_grid (dict): Diccionario de parámetros a probar en la búsqueda en cuadrícula.
    - X_train (array-like): Datos de entrenamiento.
    - y_train (array-like): Etiquetas de entrenamiento.
    - pca_components (list, opcional): Lista de componentes principales (PCA) a probar.
      Predeterminado: [0.50, 0.99]
    - cv (int, opcional): Número de divisiones para la validación cruzada. Predeterminado: 2.

    Retorna:
    tuple: Una tupla que contiene los mejores parámetros, la mejor puntuación,
    el mejor estimador y los resultados de la validación cruzada.
    """
    # Configura el pipeline con las funciones de PCA y el modelo de aprendizaje automático
    pipeline = Pipeline([
        ('pca', PCA()),
        ('classify', model)
    ])

    # Realiza la búsqueda en cuadrícula con validación cruzada utilizando el pipeline anterior
    # y los parámetros del modelo específico
    grid_search = GridSearchCV(pipeline, param_grid, cv=cv, n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Obtiene los mejores parámetros del modelo específico
    best_params = grid_search.best_params_

    # Elimina el prefijo 'classify__' de todas las claves
    best_params = {key.replace('classify__', '', 1): value for key, value in best_params.items()}

    return best_params

In [ ]:
def train(model, best_params=None):
    """
    Entrena un modelo de aprendizaje automático, opcionalmente utilizando los mejores
    parámetros obtenidos de una búsqueda en cuadrícula (grid search).

    Parámetros:
    - model: El modelo de aprendizaje automático a entrenar.
    - best_params (dict, opcional): Diccionario con los mejores parámetros para el modelo.
      Si no se proporciona, se entrenará el modelo con los parámetros predeterminados.

    Retorna:
    - El modelo entrenado.
    """
    # Si se proporcionan los mejores parámetros, se establecen en el modelo
    if best_params is not None:
        model.set_params(**best_params)

    # Entrena el modelo con los datos de entrenamiento
    return model.fit(X_train, y_train)

In [ ]:
def test(model, is_classification=False):
    """
    Evalúa el rendimiento de un modelo de aprendizaje automático en un conjunto de prueba.
    Puede manejar tanto modelos de clasificación como de regresión.

    Parámetros:
    - model: El modelo de aprendizaje automático a evaluar.
    - is_classification (bool, opcional): Indica si el modelo es de clasificación.
      Predeterminado: False (regresión).

    Retorna:
    - y_pred: Las predicciones del modelo en el conjunto de prueba.
    """
    # Realiza las predicciones del modelo en el conjunto de prueba
    y_pred = model.predict(X_test)

    # Si el modelo es de clasificación, calcula y muestra la precisión
    if is_classification:
        acc = accuracy_score(y_test, y_pred)
        print(f"Precisión del modelo: {acc}")
    # Si el modelo es de regresión, calcula y muestra el error cuadrático medio y el R²
    else:
        mse = mean_squared_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        print(f'Error Cuadrático Medio en el conjunto de prueba: {mse:.2f}')
        print(f'Coeficiente de Determinación (R²): {r2:.2f}')

    return y_pred

### Separacion de datos

In [ ]:
# separo las las variables de la variable target
X = df_estandarizado_escalado.drop(columns=['subscribed_encoded'])
y = df_estandarizado_escalado['subscribed_encoded']
# Divido las variable target y el resto de variables en testeo y prueba utilizando un 35% de datos para prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, random_state=42)

### Instancia y evaluacion de modelos

**Modelo de bosques aleatorios:**

In [ ]:
# creo instancia del modelo random forest
random_forest_model = RandomForestClassifier()

# Parametros y rango de valores para el modelo de random forest
random_forest_params_grid = {
    'classify__max_depth': [None, 5, 10],
    'classify__min_samples_split': [2, 5, 10],
    'classify__min_samples_leaf': [1, 2, 4],
    'classify__ccp_alpha': [0.0, 0.1, 0.2]
}

In [ ]:
# obtengo los mejores parametros desde grid search
random_forest_best_params_grid = grid_search_function(model= random_forest_model, param_grid=random_forest_params_grid, X_train=X_train, y_train=y_train)

In [ ]:
# entreno el modelo con los mejores parametros previamente obtenidos
random_forest_model = train(random_forest_model, random_forest_best_params_grid)

In [ ]:
# obtengo las predicciones del modelo
y_pred = test(random_forest_model, is_classification=True)

Precisión del modelo: 0.8983768035516093


**Modelo de regresion logistica:**

In [ ]:
# creo una instancia del modelo de regresion logistica
logistic_regression_model = LogisticRegression()

In [ ]:
# defino el rango de valores para evaluar los parametros del modelo
logistic_regression_params_grid = {
    'classify__penalty': ['l1', 'l2'],
    'classify__C': [0.1, 1, 10],
    'classify__fit_intercept': [True, False],
}

In [ ]:
# obtengo los mejores parametros para el modelo
logistic_regression_best_params_grid = grid_search_function(logistic_regression_model, logistic_regression_params_grid, X_train=X_train, y_train=y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
12 fits failed out of a total of 24.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 427, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3

In [ ]:
# entreno el modelo con los mejores parametros obtenidos
logistic_regression_model = train(logistic_regression_model, logistic_regression_best_params_grid)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# evaluo la presicion del modelo de regresion logistica
y_test = test(logistic_regression_model, is_classification=True)

Precisión del modelo: 0.8979605993340732


### Guardado del mejor modelo

In [ ]:
# Guardar el modelo entrenado en un archivo .pkl
path_best_ml_model = '/content/models/random_forest_model_subscribed.pkl'
joblib.dump(random_forest_model, path_best_ml_model)

['/content/models/random_forest_model_subscribed.pkl']

### Observaciones de esta etapa:

El modelo que presenta el mayor porcentaje de precisión es el de **Bosques Aleatorios, con una precisión del 89.83%** para la variable objetivo subscribed.